1. Parse HMDB json file

In [ ]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from molmass import Formula
from rdkit import Chem
from rdkit.Chem import AllChem
from PyCFMID.PyCFMID import cfm_predict
from DIA.utils import get_decoy_spectrum

In [ ]:
metabolite_json = 'HMDB/urine_metabolites.json'

with open(metabolite_json, 'r') as read_file:
    metabo_data = json.load(read_file)

hmdb, metab, smiles, precursors = [], [], [], []
for item in metabo_data:
    # combine isomers as their spectra are very similar
    try:
        precursor = Formula(item['chemical_formula']).isotope.mass + 1.0078
    except:
        precursor = np.nan
    
    try:
        smi = Chem.MolToSmiles(Chem.MolFromSmiles(item['smiles']), isomericSmiles=False)
    except:
        smi = item['smiles']
    
    if smi not in smiles:
        precursors.append(precursor)
        metab.append(item['name'])
        hmdb.append(item['accession'])
        smiles.append(item['smiles'])

hmdb_metab = pd.DataFrame({'ID': hmdb, 'Metabolite': metab, 'Adduct': '[M+H]+', 'Precursor_mz': precursors, 'SMILES': smiles})
hmdb_metab.to_csv('HMDB/urine_metabolites.csv', index = False)

2. Generate predicted spectrum and decoy spectrum

In [ ]:
spectra = []
for smi in tqdm(smiles):
    try:
        spectrum = cfm_predict(smi, param_file='', config_file='', annotate_fragments=False, output_file=None, apply_postproc=True, suppress_exceptions=False)
    except:
        spectrum = None
    spectra.append(spectrum)
np.save('HMDB/urine_spectra.npy', spectra)

In [ ]:
true_spectra, decoy_spectra = dict(), dict()
for i in tqdm(range(len(all_spectra))):
    metabolite = hmdb_metab.loc[i, 'Metabolite']
    precursor_mz = hmdb_metab.loc[i, 'Precursor_mz']
    
    if np.isnan(precursor_mz):
        true_spectra[metabolite] = spectra[i]['medium_energy']
        decoy_spectra[metabolite] = None
    else:
        decoy_spectrum = get_decoy_spectrum(precursor_mz, spectra[i]['medium_energy'])
        true_spectra[metabolite] = spectra[i]['medium_energy']
        decoy_spectra[metabolite] = decoy_spectrum
        
np.save('HMDB/true_urine_spectra.npy', true_spectra)
np.save('HMDB/decoy_urine_spectra.npy', decoy_spectra)
new_metab.to_csv('HMDB/all_metabolites.csv', index = False)

3. Process DIA dataset

In [ ]:
from DIA.core import process_dataset, grouping_results
from DIA.iq import create_metabo_list, create_metabo_table

In [ ]:
file_dir = 'D:/data/MTBLS816_mzML'
file_met = 'HMDB/all_metabolites.csv'
file_spectra = 'HMDB/true_urine_spectra.npy'
decoy_spectra = 'HMDB/decoy_urine_spectra.npy'

# true results
results, spectra = process_dataset(file_dir, file_met, file_spectra, parallel=True, energy = 30, peak_threshold=5000)
results = grouping_results(results, n_candidate=1000, rt_tol = 15)
quant_list = create_metabo_list(results, median_normalization = False, missing_value_filter = 0.3)
quant_table = create_metabo_table(quant_list, spectra, 'topN', 5)
np.save('quant_table.npy', quant_table)
np.save('quant_list.npy', quant_list)

# decoy results
decoy, decoyspectra = process_dataset(file_dir, file_met, decoy_spectra, parallel=True, energy = 30, peak_threshold=5000)
decoy = grouping_results(decoy, n_candidate=1000, rt_tol = 15)
decoy_list = create_metabo_list(decoy, median_normalization = False, missing_value_filter = 0.3)
decoy_table = create_metabo_table(decoy_list, decoyspectra, 'topN', 5)
np.save('decoy_table.npy', decoy_table)
np.save('decoy_list.npy', decoy_list)

4. Evaluate FDR

In [ ]:
n_metabolites = np.unique([i.split('_')[0] for i in quant_list.keys()])
print(len(n_metabolites))

quant_list = np.load('quant_list.npy', allow_pickle=True).item()
quant_table = np.load('quant_table.npy', allow_pickle=True)
decoy_table = np.load('decoy_table.npy', allow_pickle=True)

true_scores = np.array(quant_table[1])
decoy_scores = np.array(decoy_table[1])
decoy_scores[np.isnan(decoy_scores)] = 0

pvals = stats.t.sf((true_scores - np.mean(decoy_scores)) / np.std(decoy_scores), len(decoy_scores)-1)
thres = true_scores[np.argmin(np.abs(pvals - 0.05))]

quant_output = quant_table[0]
quant_output['MCI Score'] = quant_table[1]
quant_output['RT'] = [i.split('_')[-1] for i in quant_table[0].index]
quant_output['p value'] = pvals
quant_output.to_csv('quant_output.csv')

decoy_output = decoy_table[0]
decoy_output['MCI Score'] = decoy_table[1]
decoy_output['RT'] = [i.split('_')[-1] for i in decoy_table[0].index]
decoy_output.to_csv('decoy_output.csv')

plt.figure(dpi = 300)
plt.hist(true_scores, bins = 50, color='coral', alpha=0.7, label = 'urine')
plt.hist(decoy_scores, bins = 50, color='navy', alpha=0.7, label = 'decoy')
plt.plot([thres, thres], [0, 1200], color='red', label='p-val = 0.05')
plt.xlabel('MCI scores')
plt.ylabel('peak groups')
plt.legend()
plt.show()